## Setup

In [18]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [19]:
import pandas as pd
import numpy as np
import dill 
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.sequence_ranker.model import SequenceRatingPrediction
from src.algo.sequence_ranker.dataset import UserItemBinaryRatingDFDataset
from src.algo.sequence_ranker.trainer import SeqModellingLitModule
from src.algo.item2vec.trainer import LitSkipGram
from src.algo.item2vec.model import SkipGram
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics
from src.utils.data_prep import chunk_transform

In [20]:
load_dotenv(override = True)

True

In [21]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "pruning-sequence-modelling"
    notebook_persit_dp: str = None
    
    run_name: str = None

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "seq-modelling"
    item_metadata_pipeline_fp: str = "../data_for_ai/interim/item_metadata_pipeline_wo_user_item_manipulate.dill"
    tfm_chunk_size: int = 1000

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 512
    learning_rate: float = 0.001
    l2_reg: float = 1e-6
    early_stopping_patience: int = 5
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    max_epochs: int = 100

    # TwoTower specific
    dropout: float = 0.3
    embedding_dim: int = 256
    use_user_embedding: bool = True
    user_embedding_dim: int = 32

    use_title: bool = True
    title_embedding_dim: int = 384
    title_fc_dim: int = 32

    use_extra_item_embedding: bool = True

    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_features_neg_seq_without_stats_item_user.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_features_neg_seq_without_stats_item_user.parquet")

    best_checkpoint_path: str = None
    def init(self):
        self.run_name: str = f"(3)(8)(17) tune params encoder transformer head=8"
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=False,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-07-02 01:04:41.645 | INFO     | __main__:init:52 - Setting up Mlflow experiment: pruning-sequence-modelling, run_name: (3)(8)(17) tune params encoder transformer head=8


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "pruning-sequence-modelling",
  "notebook_persit_dp": "c:\\Users\\Trieu\\OneDrive\\Desktop\\recsys\\real_time_recsys\\notebooks\\data\\pruning-sequence-modelling\\(3)(8)(17) tune params encoder transformer head=8",
  "run_name": "(3)(8)(17) tune params encoder transformer head=8",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "seq-modelling",
  "item_metadata_pipeline_fp": "../data_for_ai/interim/item_metadata_pipeline_wo_user_item_manipulate.dill",
  "tfm_chunk_size": 1000,
  "top_K": 100,
  "top_k": 10,
  "batch_size": 512,
  "learning_rate": 0.001,
  "l2_reg": 1e-6,
  "early_stopping_patience": 5,
  "device": "cpu",
  "max_epochs": 100,
  "dropout": 0.3,
  "embedding_dim": 256,
  "use_user_embedding": true,
  "user_embedding_dim": 32,
  "use_title": true,
  "title_embedding_dim": 384,
  "title_fc_dim": 32,
  "use_extra_item_embedding":

## Init model

In [22]:
with open(args.item_metadata_pipeline_fp, "rb") as f:
    item_metadata_pipeline = dill.load(f)

In [23]:
item_metadata_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(force_int_remainder_cols='deprecated',
                                   transformers=[('main_category',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['main_category']),
                                                 ('title',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(fill_value='',
                                                                                 strategy='constant')),
                                                                  ('reshape',
                                                                   FunctionTransformer(func=<function reshape_2d_to_1d at 0x0000023FBFFC4AE0>)),
                                                                  ('...
                                                                   FunctionTransformer(func=<function todense at 0x0000023FBFFC74C0>))]),
                                                  'categories'),
                                                 ('price',
                                                  Pipeline(steps=[('extract_price',
                                                                   FunctionTransformer(func=<function price_parse_dtype at 0x0000023FBFFC7920>,
                                                                                       kw_args={'pattern': '\\b((?:\\d+\\.\\d*)|(?:\\d+))\\b'})),
                                                                  ('impute',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy='constant')),
                                                                  ('min_max_scale',
                                                                   MinMaxScaler())]),
                                                  'price')])),
                ('normalizer', StandardScaler())])

In [24]:
# Load title embedding
title_embedding_matrix = np.load("../data_for_ai/interim/metadata_title_embedding.npy")  # shape (4817, 898)
# add 2 more vector for padding and start token 
title_embedding_matrix = np.concatenate(
    [np.zeros((2, title_embedding_matrix.shape[1])), title_embedding_matrix], axis=0
)
title_embedding_tensor = torch.tensor(title_embedding_matrix, dtype=torch.float32)
title_embedding_layer = nn.Embedding.from_pretrained(
    embeddings=title_embedding_tensor,
    freeze=True,)

In [25]:
print(f"Title embedding shape: {title_embedding_tensor.shape}")

Title embedding shape: torch.Size([4819, 384])


In [26]:
def init_model(n_users, n_items, embedding_dim, dropout, item_embedding=None,
               user_embedding_dim=None, use_user_embedding=False,
               use_title=False, title_embedding=None,
               title_embedding_dim=898, title_fc_dim=256,
               use_extra_item_embedding=False, extra_item_embedding=None, extra_item_embedding_dim=None):
    return SequenceRatingPrediction(
        item_embedding=item_embedding,
        num_users=n_users,
        num_items=n_items,
        embedding_dim=embedding_dim,
        dropout=dropout,
        user_embedding_dim=user_embedding_dim,
        use_user_embedding=use_user_embedding,
        use_title=use_title,
        title_embedding=title_embedding,
        title_embedding_dim=title_embedding_dim,
        title_fc_dim=title_fc_dim,
        use_extra_item_embedding=use_extra_item_embedding,
        extra_item_embedding_dim=extra_item_embedding_dim,
    )

## Test implementation

In [27]:
embedding_dim = 32
user_embedding_dim = 64
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 3, 1, 3, 0]
# item_sequences = [
#     [2, 3, -1, -1],
#     [2, 4, -1, -1],
#     [1, 3, -1, -1],
#     [2, 1, -1, -1],
#     [4, 1, -1, -1],
# ]

item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 4],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 4, 1],
]


n_users = len(set(user_indices))

n_items = len(set(item_indices))

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
    }
)
# Mock title embedding (giả sử mỗi item có 32 chiều title embedding)
mock_title_embedding_matrix = np.random.randn(n_items, embedding_dim).astype(np.float32)
mock_title_embedding_layer = nn.Embedding.from_pretrained(
    embeddings=torch.tensor(mock_title_embedding_matrix),
    freeze=True)

model = init_model(n_users, 
                   n_items, 
                   embedding_dim, 
                   args.dropout,
                   user_embedding_dim=user_embedding_dim,
                   use_user_embedding=args.use_user_embedding,
                   use_title=args.use_title,
                   title_embedding=mock_title_embedding_layer,
                   title_embedding_dim=embedding_dim,
                   title_fc_dim=16,)

# Example forward pass
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-0, 1, -1, -1]])
target_item = torch.tensor([2])
target_title = torch.tensor([2])  # giả sử title index trùng với item index
predictions = model(user, item_sequence, target_item, target_title)
print(predictions)
model.train()

c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\torch\nn\modules\transformer.py:385: UserWarning:

enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.activation_relu_or_gelu was not True

2025-07-02 01:04:43.712 | INFO     | src.algo.sequence_ranker.model:__init__:50 - Start token used: 4, Padding token used: 5


tensor([[0.0374]], grad_fn=<MaskedFillBackward0>)


SequenceRatingPrediction(
  (item_embedding): Embedding(6, 32, padding_idx=5)
  (encoder_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
    (linear1): Linear(in_features=32, out_features=32, bias=True)
    (dropout): Dropout(p=0.3, inplace=False)
    (linear2): Linear(in_features=32, out_features=32, bias=True)
    (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.3, inplace=False)
    (dropout2): Dropout(p=0.3, inplace=False)
    (activation): PReLU(num_parameters=1)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_feat

In [28]:
rating_dataset = UserItemBinaryRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col,"item_sequence",
)

train_loader = DataLoader(
    rating_dataset, batch_size=batch_size, shuffle=False, drop_last=True
)

In [29]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  4]], dtype=torch.int32), 'item_feature': tensor([], size=(2, 0))}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1., 1.]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]], dtype=torch.int32), 'item_feature': tensor([], size=(2, 0))}


In [30]:
# model
lit_model = SeqModellingLitModule(model, log_dir=args.notebook_persit_dp)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persit_dp}/test",
    max_epochs=100,
    accelerator=args.device if args.device else "auto",
)
# trainer.fit(
#     model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
# )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [31]:
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-1, -1, -1, 0, 1]])
target_item = torch.tensor([2])
target_title = torch.tensor([2])  # giả sử title index trùng với item index
predictions = model.predict(user, item_sequence, target_item, target_title)
print(predictions)

tensor([[0.5027]], grad_fn=<SigmoidBackward0>)


## Training loop

In [32]:
# take 1% of train data for debugging
# For debugging, we can take a small sample of the training data
train_df = pd.read_parquet(args.train_data_fp)
val_df = pd.read_parquet(args.val_data_fp)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [ ]:
user_indices = train_df["user_indice"].unique()
item_indices = train_df["item_indice"].unique()

train_item_features = chunk_transform(
    train_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
train_item_features = train_item_features.astype(np.float32)

val_item_features = chunk_transform(
    val_df, item_metadata_pipeline, chunk_size=args.tfm_chunk_size
)
val_item_features = val_item_features.astype(np.float32)

logger.info(f"{len(user_indices)=:,.0f}, {len(item_indices)=:,.0f}")

2025-07-02 01:05:03.100 | INFO     | __main__:<module>:14 - len(user_indices)=16,407, len(item_indices)=4,817


In [ ]:
print(f"Train item features shape: {train_item_features.shape}")

In [ ]:
train_df

In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(train_df.loc[train_df["user_id"] == "AEEV5YWQKPBTLFWHKOBBULYA2RDQ"].sort_values(by=args.timestamp_col, ascending=False))

## Convert user_id and item_id into indices

In [ ]:
# idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
# idm = IDMapper().load(idm_path)
# idm.get_user_id(1)

In [ ]:
# train_df = train_df.pipe(idm.map_indices)
# val_df = val_df.pipe(idm.map_indices)

# assert idm.unknown_item_index not in train_df["item_indice"].values, "Unknown item index must be present in training data."
# assert idm.unknown_user_index not in train_df["user_indice"].values, "Unknown user index must be present in training data."
# assert idm.unknown_item_index not in val_df["item_indice"].values, "Unknown item index must be present in validation data."
# assert idm.unknown_user_index not in val_df["user_indice"].values, "Unknown user index must be present in validation data."

In [ ]:
# train_df.head(3)

In [ ]:
# assert train_df.groupby(args.user_col)[args.item_col].nunique().min() >= 5, "Each user must have at least five items."
# assert train_df.groupby(args.item_col)[args.user_col].nunique().min() >= 10, "Each item must have at least ten users."

## Training loop

In [ ]:
rating_dataset = UserItemBinaryRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence",item_feature=train_item_features,
)
val_rating_dataset = UserItemBinaryRatingDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence",item_feature=val_item_features,
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

assert isinstance(train_loader.dataset, UserItemBinaryRatingDFDataset), "train_loader must use UserItemBinaryRatingDFDataset"
assert isinstance(val_loader.dataset, UserItemBinaryRatingDFDataset), "val_loader must use UserItemBinaryRatingDFDataset"

## Load the weight from SkipGram Model

In [37]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")

assert args.embedding_dim == 256, "Embedding dimension must be 256"
best_trainer = LitSkipGram.load_from_checkpoint(
    "../data_for_ai/interim/best-item2vec-weight.ckpt",
    skipgram_model=SkipGram(n_items, args.embedding_dim).to(args.device),
)
skipgram_item_embedding = best_trainer.skipgram_model.embeddings.weight.data.cpu()
print(f"SkipGram Item embedding shape: {skipgram_item_embedding.shape}")
print(f"SkipGram Item embedding dtype: {skipgram_item_embedding.dtype}")
# convert skipgram_item_embedding into torch.nn.Embedding and take the first n_items rows
skipgram_item_embedding = torch.nn.Embedding.from_pretrained(
    skipgram_item_embedding[:n_items], freeze=False
)


2025-07-02 01:08:16.686 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817
2025-07-02 01:08:16.686 | INFO     | src.algo.item2vec.model:__init__:12 - Initializing item embeddings with num items 4817, embedding dim 256


SkipGram Item embedding shape: torch.Size([4818, 256])
SkipGram Item embedding dtype: torch.float32


c:\Users\Trieu\AppData\Local\pypoetry\Cache\virtualenvs\hm-scalablerecs-QHnDFvap-py3.11\Lib\site-packages\lightning\pytorch\utilities\migration\utils.py:56: PossibleUserWarning:

The loaded checkpoint was produced with Lightning v2.5.2, which is newer than your current Lightning version: v2.5.0



In [ ]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

## Overfit 1 batch

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout,item_embedding=skipgram_item_embedding,
                   user_embedding_dim=args.user_embedding_dim,
                   use_user_embedding=args.use_user_embedding,
                   use_title=args.use_title,
                   title_embedding=title_embedding_layer,
                   title_embedding_dim=args.title_embedding_dim,
                   title_fc_dim=args.title_fc_dim,
                   use_extra_item_embedding=args.use_extra_item_embedding,)

lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping],
)
# trainer.fit(
#     model=lit_model,
#     train_dataloaders=train_loader,
#     val_dataloaders=train_loader,
# )
# logger.info(f"Logs available at {trainer.log_dir}")

In [ ]:
for i in train_loader:
    print(i)
    break

## Run on all data

In [ ]:
import lightning
print(lightning.__version__)

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_roc_auc", patience=args.early_stopping_patience, mode="max", verbose=False, min_delta=0.001
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persit_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_roc_auc",
    mode="max",
)

model = init_model(n_users, n_items, 
                   args.embedding_dim, 
                   args.dropout, 
                   item_embedding=skipgram_item_embedding,
                   user_embedding_dim=args.user_embedding_dim,
                   use_user_embedding=args.use_user_embedding,
                   use_title=args.use_title,
                   title_embedding=title_embedding_layer,
                   title_embedding_dim=args.title_embedding_dim,
                   title_fc_dim=args.title_fc_dim,
                   use_extra_item_embedding=True,
                   extra_item_embedding_dim=train_item_features.shape[1],)

print(f"Model: {model}")
lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    logger=args._mlf_logger if args.log_to_mlflow else None,
    # limit_train_batches=0.1    
)

trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    
)

# Change the library as a workaround for the issue in the latest Lightning release
#https://github.com/Lightning-AI/pytorch-lightning/pull/20669/commits/429f732a0528c558e701da7ec01e51c1e2e4f32e

## Log metrics

In [ ]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = SeqModellingLitModule.load_from_checkpoint(
    checkpoint_path=checkpoint_callback.best_model_path,
    model=init_model(n_users, n_items, 
                   args.embedding_dim, 
                   args.dropout, 
                   item_embedding=skipgram_item_embedding,
                   user_embedding_dim=args.user_embedding_dim,
                   use_user_embedding=args.use_user_embedding,
                   use_title=args.use_title,
                   title_embedding=title_embedding_layer,
                   title_embedding_dim=args.title_embedding_dim,
                   title_fc_dim=args.title_fc_dim,
                   use_extra_item_embedding=True,
                   extra_item_embedding_dim=train_item_features.shape[1],),
)

In [42]:
best_trainer = SeqModellingLitModule.load_from_checkpoint(
    checkpoint_path="C:/Users/Trieu/Downloads/best-checkpoint (3).ckpt",
    model=init_model(n_users, n_items, 
                   args.embedding_dim, 
                   args.dropout, 
                   item_embedding=skipgram_item_embedding,
                   user_embedding_dim=args.user_embedding_dim,
                   use_user_embedding=args.use_user_embedding,
                   use_title=args.use_title,
                   title_embedding=title_embedding_layer,
                   title_embedding_dim=args.title_embedding_dim,
                   title_fc_dim=args.title_fc_dim,
                   use_extra_item_embedding=True,
                   extra_item_embedding_dim=288,
                   ),
)

2025-07-02 01:10:41.234 | INFO     | src.algo.sequence_ranker.model:__init__:50 - Start token used: 4816, Padding token used: 4817


RuntimeError: Error(s) in loading state_dict for SeqModellingLitModule:
	Missing key(s) in state_dict: "model.extra_item_feature_layer.0.running_mean", "model.extra_item_feature_layer.0.running_var", "model.extra_item_feature_layer.2.bias", "model.extra_item_feature_layer.2.running_mean", "model.extra_item_feature_layer.2.running_var", "model.extra_item_feature_layer.3.weight". 
	Unexpected key(s) in state_dict: "model.extra_item_feature_layer.1.running_mean", "model.extra_item_feature_layer.1.running_var", "model.extra_item_feature_layer.1.num_batches_tracked". 
	size mismatch for model.extra_item_feature_layer.0.weight: copying a param with shape torch.Size([256, 1]) from checkpoint, the shape in current model is torch.Size([256]).
	size mismatch for model.extra_item_feature_layer.1.weight: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([256, 288]).
	size mismatch for model.extra_item_feature_layer.2.weight: copying a param with

In [ ]:
best_model = best_trainer.model.to(args.device)
best_model.eval()

In [ ]:
val_recs_df = val_df.sort_values(by=args.timestamp_col).drop_duplicates(subset=[args.user_col], keep="first")

In [ ]:
mlflow.start_run(run_id = trainer.logger.run_id)

### Classification metrics

In [ ]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values
val_item_sequences = val_df["item_sequence"].values.tolist()
# take item features for the items in val_features
val_item_features = val_item_features[val_item_indices]

In [ ]:
users = torch.tensor(val_user_indices, device=args.device)
item_sequences = torch.tensor(val_item_sequences, device=args.device)
items = torch.tensor(val_item_indices, device=args.device)
item_embeddings = torch.tensor(val_item_features, device=args.device)
classifications = best_model.predict(users, item_sequences, items, item_embeddings=item_embeddings)

In [ ]:
classifications.shape

In [ ]:
eval_classification_df = val_df.assign(
    classification_proba=classifications.cpu().detach().numpy(),
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)

In [ ]:
eval_classification_df.head(3)

In [ ]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

### Ranking metrics

In [ ]:
val_recs_df

In [ ]:
# sort the train_df by timestamp and get the lastest item features from train_df
all_items_df = train_df.sort_values(by=args.timestamp_col, ascending=False)
# get the lastest item features from train_df
all_items_indices = all_items_df.drop_duplicates(subset=[args.item_col], keep="first")["item_indice"].values
all_items_features = item_metadata_pipeline.transform(all_items_df).astype(np.float32)
logger.info(
    f"Mean std over categorical and numerical features: {all_items_features.std(axis=0).mean()}"
)

In [ ]:
recommendations = best_model.recommend(
    torch.tensor(val_recs_df["user_indice"].values, device=args.device),
    torch.tensor(val_recs_df["item_sequence"].values.tolist(), device=args.device),
    all_items_indices=torch.tensor(all_items_indices, device=args.device),
    all_items_features=torch.tensor(all_items_features, device=args.device),
    k=args.top_K,
    batch_size=32)

In [ ]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

In [ ]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

In [ ]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

In [ ]:
ranking_report = log_ranking_metrics(args, eval_df)

In [ ]:
mlflow.end_run()

## Clean up

In [ ]:
all_params = [args]

if args.log_to_mlflow:
    run_id = trainer.logger.run_id

    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.model_dump()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)